<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️ תהליך הסבת שכבת תשריטי תיעוד גבולות לבנק"ל מודרני 🗺️ 

In [ ]:
print('6. Tatag borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
from configs import CNFG
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  6.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תת"ג קיימת

In [2]:
tatag_cols = ['TATAG_NUM', 'TATAG_YEAR', 'TATAG_STATUS', 'SURVEYOR_NO', 'GUSH_NUM', 'GUSH_SUFFIX', 'geometry']

tatag_layer = gpd.read_file(CNFG.CADSDE + 'TATAG.gdb', layer = 'TATAG', include_fields = tatag_cols)

<h1 style="text-align:right;font-size:100%">
שכבת גושי קדסטר מודרנית

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'BlockUniqueID'])

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  6.b Cleaning data')

In [ ]:
print('  6.c Data conversion')

<h1 style="text-align:right;font-size:125%">
:עיבוד נתוני תת"ג
<h1 style="text-align:right;font-size:100%">
הסרת מספרי מלבן שאינם תקינים
<h1 style="text-align:right;font-size:100%">
הסרת רשומות ללא מידע של גוש    
<h1 style="text-align:right;font-size:100%">
מילוי ערכי מספרי תת-גוש ב-0    
<h1 style="text-align:right;font-size:100%">
הוספת שדה מקשר של שם מלא לגוש לטובת איחוד עם שכבת גושים וקבלת מספר מזהה לגוש

In [ ]:
legit_rectangle_min = 1
legit_rectangle_max = 4000

tatag_layer = tatag_layer[tatag_layer['TATAG_NUM'].between(legit_rectangle_min, legit_rectangle_max)]

del [legit_rectangle_min, legit_rectangle_max]

tatag_layer.dropna(subset = ['GUSH_NUM'], inplace=True)

tatag_layer['GUSH_NUM'] = tatag_layer['GUSH_NUM'].astype(int)
tatag_layer['GUSH_SUFFIX'] = tatag_layer['GUSH_SUFFIX'].fillna(0)
tatag_layer['GUSH_SUFFIX'] = tatag_layer['GUSH_SUFFIX'].astype(int)
tatag_layer['BlockName'] = tatag_layer['GUSH_NUM'].astype(str) + '/' + tatag_layer['GUSH_SUFFIX'].astype(str)

In [ ]:
tatag_layer.dropna(subset = ['GUSH_NUM'], inplace=True)

tatag_layer['GUSH_NUM'] = tatag_layer['GUSH_NUM'].astype(int)
tatag_layer['GUSH_SUFFIX'] = tatag_layer['GUSH_SUFFIX'].fillna(0)
tatag_layer['GUSH_SUFFIX'] = tatag_layer['GUSH_SUFFIX'].astype(int)
tatag_layer['BlockName'] = tatag_layer['GUSH_NUM'].astype(str) + '_' + tatag_layer['GUSH_SUFFIX'].astype(str)

tatag_layer = tatag_layer.merge(Blocks, left_on = 'BlockName', right_on='Name', how='left')

In [ ]:
tatag_layer['ProcessName'] = tatag_layer['TATAG_NUM'].astype('str') +'/'+ tatag_layer['TATAG_YEAR'].astype('str')

duplicated_names = ['10/2014', '1335/2018', '1784/2015', '182/2014', '2/2013', '21/2014', '22/2014', '221/2020', '2232/2019',
                    '23/2014', '2383/2022', '24/2014', '25/2014', '2585/2020', '3/2013', '4/2013', '455/2020', '5/2013',
                    '500/2017', '692/2014', '933/2019']

tatag_layer['ProcessName'] = np.where(tatag_layer['ProcessName'].isin(duplicated_names),
                                      '0' + tatag_layer['ProcessName'],
                                      tatag_layer['ProcessName'])

del duplicated_names

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  6.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                       ("BlockUniqueID", int),
                        ("ProcessType", int),
                        ("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),
                        ("Status", int),
                        #("PlanName", str)
                        ("SurveyorLicenseID", int),
                        ("DataSource", int),
                        ("geometry", str)
                        ])


tatag_new = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                             geometry = 'geometry',
                             crs = 2039)

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
tatag_new['MapYear'] = tatag_layer['TATAG_YEAR']

#tatag_new['PlanName'] = None

tatag_new['ProcessName'] = tatag_layer['ProcessName']

tatag_new['geometry'] = tatag_layer['geometry']

tatag_new['ProcessType'] = 4

tatag_new['DataSource'] = 5

statusDict = {0 : 2,     #לא ידוע = בביקורת
              1 : 2,     #בביקורת = בביקורת
              2 : 13,    #מאושר = מאושר
              3 : 9,     #מחיקת תת"ג = מבוטל
              None : 2}  #לא ידוע = בביקרות

tatag_new['Status'] = tatag_layer['TATAG_STATUS'].map(statusDict)


geodeticNetDict = {range(0,    1998)  : 1,     # רשת ישראל הישנה
                   range(1999, 2007)  : 2,     # רשת ישראל החדשה
                   range(2008, 99999) : 3}     # רשת ישראל התקפה
tatag_new['GeodeticNetwork'] = tatag_new['MapYear'].apply(lambda x: next((v for k, v in geodeticNetDict.items() if x in k), 0))


tatag_new['SurveyorLicenseID'] = tatag_layer['SURVEYOR_NO']
tatag_new['SurveyorLicenseID'] = tatag_new['SurveyorLicenseID'].astype(int)


tatag_new['BlockUniqueID'] = tatag_layer['BlockUniqueID']
tatag_new['BlockUniqueID'] = tatag_new['BlockUniqueID'].fillna(0)
tatag_new['BlockUniqueID'] = tatag_new['BlockUniqueID'].astype(int)


tatag_new = tatag_new.drop_duplicates(subset='ProcessName', ignore_index=True, keep='last')


#הסרת תכניות בעלי סטאטוס לא ידוע
tatag_new.dropna(inplace=True, subset=['Status'])


tatag_new['CPBUniqueID'] = range(1, 1 + len(tatag_new))
tatag_new['CPBUniqueID'] = tatag_new['CPBUniqueID'] + 4000000

In [ ]:
tatag_new.sort_values(['CPBUniqueID', 'MapYear'],inplace = True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  6.e Exporting results to product folder')

In [ ]:
json_dir =  CNFG.JsonPath + 'Tatag.geojson'

tatag_new.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + 'Tatag-Save as Feature Class.bat']);